In [6]:
# Importing required modules
import pandas as pd
import os
import re
import pyodbc as odbc;

Enable variables for connecting to server
1. since we are using windows authentication in sql server Trusted_connection would enable connection
2. if wanted to connect to different sql server then use username and password

In [7]:
"""Connects to a SQL database using pyodbc"""
SERVER = 'DESKTOP-5B73FCQ'
DATABASE = 'Sales'
USERNAME = 'DESKTOP-5B73FCQ\Sai Prakash'
PASSWORD = 'bannubittu'
Trusted_connection ='yes'

In [8]:
connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_connection={Trusted_connection}'

In [9]:
#create a connection varibale to store connection to sql server
conn = odbc.connect(connectionString)

In [10]:
cursor= conn.cursor()

In [11]:
sql_query="""select * from Sales.dbo.Transactions
where TransactionID=1;
             """

In [12]:
#use cursor.execute() to retrieve result against the database
cursor.execute(sql_query)

In [13]:
#retrive executed result set using fetchall() and print
records = cursor.fetchall()
for r in records:
    print(f"{r.TransactionID}\t{r.TransactionDate}\t{r.TransactionTime}\t{r.Quantity}\t{r.StoreID}\t{r.ProductID}\t{r.UnitPrice}\t{r.Revenue}")


1	2023-01-01	07:06:11	2	5	32	3.0	6.0


Defining a function for data ingestion which loads the source files in to a Data frame

In [14]:
#Definign a function for validating the ingestion
def dataingestion():
# Directory containing CSV files
    directory = r'C:\Users\Sai Prakash\Desktop\Myfiles\BI_Project\BI_Project\Data'

# List all files in the directory
    files = os.listdir(directory)

# Filter out CSV files
    csv_files = [file for file in files if file.endswith('.csv')]

    for file in csv_files:
        DF=pd.DataFrame({})
        pattern = re.compile('.*Fraudulent_E-Commerce_Transaction.*')
        if pattern.match(file):
            DF=pd.read_csv(str(directory)+'/'+str(file))
            return DF
        else: print("No CSV files availavle for ingestion")
    


In [15]:
data= dataingestion()

In [16]:
data.head()

,Transaction ID,Customer ID,Transaction Amount,Transaction Date,Payment Method,Product Category,Quantity,Customer Age,Customer Location,Device Used,IP Address,Shipping Address,Billing Address,Is Fraudulent,Account Age Days,Transaction Hour
0,15d2e414-8735-46fc-9e02-80b472b2580f,d1b87f62-51b2-493b-ad6a-77e0fe13e785,58.09,2024-02-20 05:58:41,bank transfer,electronics,1,17,Amandaborough,tablet,212.195.49.198,Unit 8934 Box 0058\nDPO AA 05437,Unit 8934 Box 0058\nDPO AA 05437,0,30,5
1,0bfee1a0-6d5e-40da-a446-d04e73b1b177,37de64d5-e901-4a56-9ea0-af0c24c069cf,389.96,2024-02-25 08:09:45,debit card,electronics,2,40,East Timothy,desktop,208.106.249.121,"634 May Keys\nPort Cherylview, NV 75063","634 May Keys\nPort Cherylview, NV 75063",0,72,8
2,e588eef4-b754-468e-9d90-d0e0abfc1af0,1bac88d6-4b22-409a-a06b-425119c57225,134.19,2024-03-18 03:42:55,PayPal,home & garden,2,22,Davismouth,tablet,76.63.88.212,"16282 Dana Falls Suite 790\nRothhaven, IL 15564","16282 Dana Falls Suite 790\nRothhaven, IL 15564",0,63,3
3,4de46e52-60c3-49d9-be39-636681009789,2357c76e-9253-4ceb-b44e-ef4b71cb7d4d,226.17,2024-03-16 20:41:31,bank transfer,clothing,5,31,Lynnberg,desktop,207.208.171.73,"828 Strong Loaf Apt. 646\nNew Joshua, UT 84798","828 Strong Loaf Apt. 646\nNew Joshua, UT 84798",0,124,20
4,074a76de-fe2d-443e-a00c-f044cdb68e21,45071bc5-9588-43ea-8093-023caec8ea1c,121.53,2024-01-15 05:08:17,bank transfer,clothing,2,51,South Nicole,tablet,190.172.14.169,"29799 Jason Hills Apt. 439\nWest Richardtown, ...","29799 Jason Hills Apt. 439\nWest Richardtown, ...",0,158,5


In [17]:
data.shape[0]

1472952

#sql alchemy is used to connect to databases and communicate in python

In [23]:
from sqlalchemy import create_engine
driver = 'ODBC Driver 17 for SQL Server'
connection_string = f'mssql+pyodbc://@{SERVER}/{DATABASE}?driver={driver}&trusted_connection=yes'
engine=create_engine(connection_string)


In [25]:
#insert the dataframe into your Stg table
# Replace 'your_table_name' with the name of the table you want to insert data into
data.to_sql('STG_Transactions', con=engine, if_exists='append', index=False)


119

#Data is insgested into stgae table successfully, Now check out on developing a store procedure to perform warehousing